In [ ]:
import os
!pip install neo4j
from neo4j import GraphDatabase

senha = ""
url = ""
usuario = "neo4j"

In [6]:
# Exemplos
# https://gist.github.com/wjgilmore/8ba5f31ef1435dc04c52

class CriarAmizades:
  def __init__(self, uri, user, password):
    self.driver = GraphDatabase.driver(uri, auth=(user, password))

  def close(self):
    self.driver.close()

  def carrega_base(self):
    with self.driver.session() as session:
      session.write_transaction(self._zera_base)
      session.write_transaction(self._cria_amizade)

  @staticmethod
  def _zera_base(tx):
    tx.run("""
MATCH (n)
DETACH DELETE n
""")

  @staticmethod
  def _cria_amizade(tx):
    tx.run("""
CREATE (jose:PESSOA {nome: 'Jose da Silva', id: 1234})
CREATE (mariana:PESSOA {nome: 'Mariana Almeida', id: 5678})
CREATE (gabriela:PESSOA {nome: 'Gabriela Oliveira', id: 9876})
CREATE (caio:PESSOA {nome: 'Caio Catalão', id: '4326'})

CREATE (a101:SALA {sala: 'A101', capacidade: 10, bloco: "A"})
CREATE (c127:SALA {sala: 'C127', capacidade: 5, bloco: "C"})
CREATE (a231:SALA {sala: 'A231', capacidade: 10, bloco: "A"})
CREATE (b222:SALA {sala: 'B222', capacidade: 8, bloco: "B"})

CREATE (ciencia:DISCIPLINA {nome: 'Ciência de dados', id: 1, créditos: 2})
CREATE (paradigmas:DISCIPLINA {nome: 'Paradigmas', id: 2, créditos: 2})
CREATE (so:DISCIPLINA {nome: 'SO', id: 3, créditos: 4})
CREATE (ia:DISCIPLINA {nome: 'IA', id: 4, créditos: 4})
CREATE (compiladores:DISCIPLINA {nome: 'Compiladores', id: 5, créditos: 4})


CREATE (jose)-[:MATRICULADO]->(ciencia)
CREATE (mariana)-[:MATRICULADO]->(ciencia)
CREATE (gabriela)-[:MATRICULADO]->(ciencia)

CREATE (jose)-[:MATRICULADO]->(paradigmas)
CREATE (mariana)-[:MATRICULADO]->(paradigmas)
CREATE (gabriela)-[:MATRICULADO]->(paradigmas)

CREATE (mariana)-[:MATRICULADO]->(ia)

CREATE (gabriela)-[:MATRICULADO]->(so)
CREATE (caio)-[:MATRICULADO]->(so)

CREATE (jose)-[:MATRICULADO]->(compiladores)
CREATE (gabriela)-[:MATRICULADO]->(compiladores)
CREATE (caio)-[:MATRICULADO]->(compiladores)

CREATE (ciencia)-[:ALOCADO]->(a101)
CREATE (paradigmas)-[:ALOCADO]->(a101)
CREATE (ia)-[:ALOCADO]->(b222)
CREATE (compiladores)-[:ALOCADO]->(a231)
CREATE (so)-[:ALOCADO]->(c127)
""")


db = CriarAmizades(url, usuario, senha)
db.carrega_base()
db.close()
print("base carregada")

<ipython-input-6-ff77a60f16e2>:13: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._zera_base)
<ipython-input-6-ff77a60f16e2>:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._cria_amizade)


base carregada


# Consultas

In [7]:
class Neo4jConnection:
  def __init__(self, uri, user, pwd):
    self.__uri = uri
    self.__user = user
    self.__pwd = pwd
    self.__driver = None
    try:
        self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
    except Exception as e:
        print("Failed to create the driver:", e)

  def close(self):
    if self.__driver is not None:
        self.__driver.close()

  def query(self, query, parameters=None, db=None):
    assert self.__driver is not None, "Driver not initialized!"
    session = None
    response = None
    try:
      session = self.__driver.session(database=db) if db is not None else self.__driver.session()
      response = list(session.run(query, parameters))
    except Exception as e:
      print("Query failed:", e)
    finally:
      if session is not None:
        session.close()
    return response

conn = Neo4jConnection(uri=url,
                       user=usuario,
                       pwd=senha)

def consultar(query):
  return conn.query(query)
  # for line in conn.query(query):
  #   print(dict(line))

In [9]:
##### Estudante + créditos totais


# Match (a:Nó)-[r:Relacao]->(b:Nó)
####

print("Créditos do estudante:\n")

consultar("""
MATCH (p:PESSOA)-[:MATRICULADO]->(d:DISCIPLINA)
WITH p.nome AS nome, SUM(d.créditos) AS total_creditos
RETURN nome, total_creditos
ORDER BY total_creditos DESC
""")

Créditos do estudante:



[<Record nome='Gabriela Oliveira' total_creditos=12>,
 <Record nome='Jose da Silva' total_creditos=8>,
 <Record nome='Mariana Almeida' total_creditos=8>,
 <Record nome='Caio Catalão' total_creditos=8>]

In [10]:
print("\nEstudantes com mais de 4 créditos:\n")

consultar("""
MATCH (p:PESSOA)-[:MATRICULADO]->(d:DISCIPLINA)
WITH p.nome AS nome, SUM(d.créditos) AS total_creditos
WHERE total_creditos > 4
RETURN nome, total_creditos
ORDER BY total_creditos DESC
""")


Estudantes com mais de 4 créditos:



[<Record nome='Gabriela Oliveira' total_creditos=12>,
 <Record nome='Jose da Silva' total_creditos=8>,
 <Record nome='Mariana Almeida' total_creditos=8>,
 <Record nome='Caio Catalão' total_creditos=8>]

In [11]:
print("\nDisciplina por aluno:\n")

consultar("""
MATCH (p:PESSOA)-[:MATRICULADO]->(d:DISCIPLINA)
WITH d.nome AS disciplina, COUNT(p) AS num_alunos
RETURN disciplina, num_alunos
ORDER BY num_alunos DESC
""")


Disciplina por aluno:



[<Record disciplina='Ciência de dados' num_alunos=3>,
 <Record disciplina='Paradigmas' num_alunos=3>,
 <Record disciplina='Compiladores' num_alunos=3>,
 <Record disciplina='SO' num_alunos=2>,
 <Record disciplina='IA' num_alunos=1>]

In [13]:
print("\nDisciplina com mais alunos matriculados ordenada:\n")

consultar("""
MATCH (d:DISCIPLINA)-[:ALOCADO]->(s:SALA)
MATCH (p:PESSOA)-[:MATRICULADO]->(d)
WITH s, COUNT(p) AS num_alunos, s.capacidade AS capacidade
WITH s.sala AS sala, num_alunos, capacidade, (toFloat(num_alunos) / capacidade) * 100 AS ocupacao
RETURN sala, capacidade, num_alunos, ocupacao
ORDER BY ocupacao DESC
""")


Disciplina com mais alunos matriculados ordenada:



[<Record sala='A101' capacidade=10 num_alunos=6 ocupacao=60.0>,
 <Record sala='C127' capacidade=5 num_alunos=2 ocupacao=40.0>,
 <Record sala='A231' capacidade=10 num_alunos=3 ocupacao=30.0>,
 <Record sala='B222' capacidade=8 num_alunos=1 ocupacao=12.5>]

In [ ]:
print("\nAlunos por sala de aula:\n")

consultar("""
MATCH (s:SALA)<-[:ALOCADO]-(d:DISCIPLINA)<-[:MATRICULADO]-(p:PESSOA)
RETURN s.sala AS Sala, COLLECT(p.nome) AS Alunos

""")


Alunos por sala de aula:



[<Record Sala='A101' Alunos=['Jose da Silva', 'Mariana Almeida', 'Gabriela Oliveira', 'Jose da Silva', 'Mariana Almeida', 'Gabriela Oliveira']>,
 <Record Sala='C127' Alunos=['Gabriela Oliveira', 'Caio Catalão']>,
 <Record Sala='B222' Alunos=['Mariana Almeida']>,
 <Record Sala='A231' Alunos=['Jose da Silva', 'Gabriela Oliveira', 'Caio Catalão']>]

In [15]:
# https://neo4j.com/docs/cypher-manual/current/syntax/patterns/#cypher-pattern-label

print("\nQuantidade de alunos por blocos da faculdade (A,B,C):\n")

consultar("""
MATCH (d:DISCIPLINA)-[:ALOCADO]->(s:SALA)
MATCH (p:PESSOA)-[:MATRICULADO]->(d)
WITH s.bloco AS bloco, COUNT(p) AS num_alunos
RETURN bloco, num_alunos
ORDER BY bloco ASC
""")


Quantidade de alunos por blocos da faculdade (A,B,C):



[<Record bloco='A' num_alunos=9>,
 <Record bloco='B' num_alunos=1>,
 <Record bloco='C' num_alunos=2>]